In [ ]:
import os
import re
import sys
import json
import numpy as np
import pandas as pd

import requests
import seaborn as sns
import matplotlib.pyplot as plt

import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter

from datetime import datetime, timedelta

In [ ]:
API_URL = "https://flowmaps.life.bsc.es/api"

def fetch_all_pages(collection, query={}, batch_size=1000, projection={}):
    base_url = API_URL
    url = f"{base_url}/{collection}"
    params = {'where': json.dumps(query), 'max_results': batch_size, 'projection': json.dumps(projection)}
    data = []
    response = requests.get(url, params=params).json() # get first page
    data.extend(response['_items'])
    if '_links' not in response:
        return data
    num_docs = response['_meta']['total']
    if num_docs <= 0:
        return data
    while 'next' in response['_links']:
        print('.', end='')
        url = f"{base_url}/{response['_links']['next']['href']}"
        response = requests.get(url).json()
        data.extend(response['_items'])
    return data


def daily_mobility_matrix(source_layer, target_layer, date):
    filters = {
        'date': date,
        'source_layer': source_layer,
        'target_layer': target_layer,
    }
    data = fetch_all_pages('mitma_mov.daily_mobility_matrix', filters)

    return pd.DataFrame(data)
    

def get_population_who_travels(layer, date):
    filters = {
        'layer': layer,
        'type': 'zone_movements',
        'date': date,
    }
    data = fetch_all_pages('layers.data.consolidated', filters)
    df = pd.DataFrame(data)
    return df[df['viajes'] != 0]['personas'].sum() / df['personas'].sum()

In [ ]:
reference_dates = ['2020-02-15', '2020-04-15', '2020-07-15', '2020-11-15']

source_layer = "cnig_ccaa"
target_layer = "cnig_ccaa"

date = reference_dates[0]

get_population_who_travels(source_layer, date)

In [ ]:
source_layer = "cnig_ccaa"
target_layer = "cnig_ccaa"

start = '2020-02-14'
end = '2021-01-01'
data = []
for d in pd.date_range(start=start, end=end):
    date = d.strftime("%Y-%m-%d") 
    df_odm = daily_mobility_matrix(source_layer, target_layer, date)
    
    inter_ccaa = df_odm['trips'][ df_odm['source'] == df_odm['target'] ].sum()
    intra_ccaa = df_odm['trips'][ df_odm['source'] != df_odm['target'] ].sum()
    pop_who_travels = get_population_who_travels(source_layer, date)

    data.append((d, date, inter_ccaa, intra_ccaa, pop_who_travels))

In [ ]:
df_trips = pd.DataFrame(data, columns=['date', 'sdate', 'intra', 'inter', 'pop_who_travels'])

In [ ]:
import datetime as dt
import matplotlib as mpl
import matplotlib.ticker as mtick

sns.set_style("whitegrid")
fig, axes = plt.subplots(3,1, figsize=(14,6), sharex=True, dpi=300)


ax = axes[0]
x = df_trips.date
y = df_trips.pop_who_travels * 100
ax.plot(x, y, c='r')
ax.ticklabel_format(axis='y', style='sci')
ax.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
ymin, ymax = ax.get_ylim()
ax.set_ylim((ymin*0.4, ymax*1.2))
ax.set_ylabel('Population')
ax.yaxis.set_major_formatter(mtick.PercentFormatter())


ax = axes[1]
x = df_trips.date
y = df_trips.inter
ax.plot(x, y, c='b')
ax.ticklabel_format(axis='y', style='sci')
ax.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
ymin, ymax = ax.get_ylim()
ax.set_ylim((ymin*0.4, ymax*1.2))
ax.set_ylabel('Nº of Trips')


ax = axes[2]
y = df_trips.intra
ax.plot(x, y, c='g')
ax.yaxis.set_major_formatter(mpl.ticker.ScalarFormatter(useMathText=True, useOffset=False))
ymin,ymax = ax.get_ylim()
ax.set_ylim((ymin*0.3, ymax*1.2))
ax.set_ylabel('Nº of Trips')


ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.set(xlabel="", xlim=[x[0], dt.datetime.strptime("2021-01-14",'%Y-%m-%d').date()])

fig.autofmt_xdate(rotation=45, ha="center")
sns.despine(fig)
fig.tight_layout()

fig.savefig('../out/Figure_8.png')